Google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


## spec

In [2]:
!nvidia-smi

Mon Sep  6 07:32:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get GitHub repository files and install python packages

In [3]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b WJ
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 255 (delta 141), reused 152 (delta 39), pack-reused 0
Receiving objects: 100% (255/255), 2.95 MiB | 14.12 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 5.4 MB/s 
     |████████████████████████████████| 282 kB 35.7 MB/s 
     |████████████████████████████████| 2.8 MB 35.8 MB/s 
     |████████████████████████████████| 919 kB 38.8 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 243 kB 46.4 MB/s 
     |████████████████████████████████| 119 kB 45.9 MB/s 
     |████████████████████████████████| 3.3 MB 36.5 MB/s 
     |████████████████████████████████| 895 kB 33.6 MB/s 
     |████████████████████████████████| 636 kB 34.2 MB/s 
     |██████████████████████████

## model and tokenizer



In [27]:
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [28]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer

# model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", **model_config)
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

loading file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/32dc9196217c0cc26c7dd705168e8615ea2d82613aa5b672d7647b8e8d58545f.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8
loading file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/a6c32c62ff893fb2aa32dabc5722c9f9eb7243cc1cb4514b9ba3fdb1b52704d6.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7
loading file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/tokenizer.json from cache at None
loading configuration file https://

## datasets

In [6]:
from data_loader import get_data_loaders

train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)

Using custom data configuration default-5097c3b37886fad9


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5097c3b37886fad9/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

## metric

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "f1": f1}

## Do Train

In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import set_seed

set_seed(42)

In [26]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=f'{path_to_drive}/koelectra-small-v3-discriminator-test',
    seed=42,
    num_train_epochs=200,
    learning_rate=1e-4,
    weight_decay=0.0,
    gradient_accumulation_steps=1,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,

    # checkpoint
    overwrite_output_dir=True,
    save_strategy='steps',
    save_steps=1000,

    # evaluation
    evaluation_strategy='steps',
    eval_steps=1000,
    metric_for_best_model="f1",

    # early stopping
    load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
from trainer import ImbalancedSamplerTrainer

def model_init():
    return ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", **model_config, return_dict=True)

trainer = ImbalancedSamplerTrainer(
    model_init=model_init,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
    # optimizers=(optimizer, scheduler)
)

loading configuration file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/bd0f09888c5a5619ddb9de81d4a9936a94e5f45064f9a23ba6d39241ceebce02.d2485d28e5c07ca60bfa4fe84af673e0df83401e5c56bcdd991878cb4966eb34
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary

# install ray

In [13]:
!pip install "ray[tune]" transformers datasets scipy sklearn torch

     |████████████████████████████████| 49.6 MB 6.1 kB/s 
     |████████████████████████████████| 72 kB 498 kB/s 
     |████████████████████████████████| 124 kB 50.0 MB/s 


# haperparameter search

In [30]:
from ray.tune.schedulers import PopulationBasedTraining

scheduler = PopulationBasedTraining(
    metric="f1",
    mode="max"
)

In [ ]:
trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    n_trials=10,
    scheduler=scheduler
)

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
You are using PopulationBasedTraining but you haven't enabled checkpointing. This means your trials will train from scratch everytime they are exploiting new configurations. Consider enabling checkpointing by passing `keep_checkpoints_num=1` as an additional argument to `Trainer.hyperparameter_search`.
(pid=699) 2021-09-06 07:39:03,351	ERROR worker.py:428 -- SystemExit was raised from the worker
(pid=699) Traceback (most recent call last):
(pid=699)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.task_execution_handler
(pid=699)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=699)   File "python/ray/_raylet.pyx", line 525, in ray._raylet.execute_task
(pid=699)   File "python/ray/_raylet.pyx", line 532, in ray._raylet.execute_task
(pid=699)   File "python/ray/_raylet.pyx", line 536, in ray._raylet.execute_task
(pid=6

== Status ==
Memory usage on this node: 2.6/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-09-06_07-39-03
Number of trials: 10/10 (10 PENDING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+----------|
| _objective_81fed_00000 | PENDING  |       |     5.61152e-06 |                  5 |                            64 |  8.15396 |
| _objective_81fed_00001 | PENDING  |       |     1.56207e-05 |                  2 |                            16 |  7.08379 |
| _objective_81fed_00002 | PENDING  |      

(pid=742) Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
(pid=742) - This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=742) - This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=742) Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3

Result for _objective_81fed_00000:
  {}
  


(pid=775) Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
(pid=775) - This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=775) - This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=775) Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3

(pid=775) {'loss': 0.9483, 'learning_rate': 1.4068557976550074e-05, 'epoch': 0.2}


 17%|█▋        | 840/5032 [09:31<47:23,  1.47it/s]
